In [1]:
%%capture
%pip install pulp

In [2]:
from pulp import *

| Biblioteca | Tempo de Execução (s) | Consumo de RAM (MB) | Saída em Disco (MB) |
|------------|------------------------|---------------------|---------------------|
| Pandas | 52.4 | 850 | 120 |
| PySpark | 46.8 | 780 | 125 |
| Polars | 31.5 | 630 | 210 |

In [3]:
modelo = LpProblem('Minimizar_Custos_lambdas', LpMinimize)

x1 = LpVariable('Pandas', lowBound=0, cat=LpInteger)
x2 = LpVariable('PySpark', lowBound=0, cat=LpInteger)
x3 = LpVariable('Polars', lowBound=0, cat=LpInteger)

### Objetivo (Minimização do Gasto gerado por cada Lambda)

Gasto Individual = 0.00000016210937 * Tempo de Execução (Ms) * Consumo de RAM (Mb)

In [4]:
modelo +=\
	(0.00000016210937 * (52.4*1000) * 850 * x1) +\
	(0.00000016210937 * (46.8*1000) * 780 * x2) +\
	(0.00000016210937 * (31.5*1000) * 630 * x3), 'Perda_Total'

### Restrições do Modelo

#### Restringindo por quantidade requisitada de chamadas

In [5]:
USOS_MENSAIS = 30

In [6]:
modelo += x1 + x2 + x3 == 30

#### Restringindo por máximo de armazenamento em disco

In [7]:
MAXIMO_DISCO_S3 = 4000

In [8]:
modelo +=\
	120 * x1 +\
	125 * x2 +\
	170 * x3 <= MAXIMO_DISCO_S3, 'Máximo_Disco_Disponivel'

#### Restringindo por número mínimo de chamadas que precisamos para gerar registros relevantes depois

In [9]:
AMOSTRA_MINIMA_POSSIVEL = 5

In [10]:
for var in [
	(x1 >= AMOSTRA_MINIMA_POSSIVEL, 'Pandas'),
	(x2 >= AMOSTRA_MINIMA_POSSIVEL, 'PySpark'),
	(x3 >= AMOSTRA_MINIMA_POSSIVEL, 'Polars'),
]:
	modelo += var

### Visualização dos resultados

In [11]:
modelo.solve(PULP_CBC_CMD(msg=False))

1

In [12]:
print(f"Quantidade de chamadas do Pandas (x1): {x1.varValue}")
print(f"Quantidade de chamadas do PySpark (x2): {x2.varValue}")
print(f"Quantidade de chamadas do Polars (x3): {x3.varValue}")

Quantidade de chamadas do Pandas (x1): 5.0
Quantidade de chamadas do PySpark (x2): 19.0
Quantidade de chamadas do Polars (x3): 6.0


In [13]:
f"Gasto Total: {modelo.objective.value()}"

'Gasto Total: 167.83928779202'